<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages-and-functions" data-toc-modified-id="Import-packages-and-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages and functions</a></span></li><li><span><a href="#Import-data-and-parameters" data-toc-modified-id="Import-data-and-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data and parameters</a></span></li><li><span><a href="#Historical-initialization" data-toc-modified-id="Historical-initialization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Historical initialization</a></span></li><li><span><a href="#Multi-Scenario-Simulations" data-toc-modified-id="Multi-Scenario-Simulations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Multi-Scenario Simulations</a></span></li><li><span><a href="#Previously-run-scenarios" data-toc-modified-id="Previously-run-scenarios-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Previously-run scenarios</a></span></li></ul></div>

# Import packages and functions

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
idx = pd.IndexSlice
import random
from gurobipy import *

from cn_mine_simulation_tools import *
from cn_refinery import *
from cn_scrap_supply_tools import *
from cn_demand_tools import *
from cn_price_formation import * 
from cn_blending import *

import warnings
warnings.filterwarnings("ignore") # to deal with pandas datetime deprecation

# Import data and parameters

In [3]:
## High level parameters
historical_prod=pd.read_excel('Data/Production data compile.xlsx', sheet_name='Selected', index_col=0).loc[:2018]
historical_prod_cn=pd.read_excel('Data/Production data compile.xlsx', sheet_name='China', index_col=0, usecols='A:O,W:X,AE').loc[:2018]
historical_prod_rw = historical_prod.loc[1950:] - historical_prod_cn.loc[1950:]
raw_price=pd.read_excel('Data/Price data compile.xlsx', sheet_name='Price', index_col=0) # All prices 2017 constant
raw_price.drop(['Grape','Low_brass','Pb_Red_Brass'], axis = 1, inplace = True)

# Specific prod and mine data
historical_mining_prod=historical_prod.loc[:, 'Total mining production']
historical_mining_prod_cn = historical_prod_cn.loc[:, 'Total mining production']
historical_mining_prod_rw = historical_prod_rw.loc[:, 'Total mining production']

## Primary supply data and patameters
operating_mine_pool=pd.read_excel('Data/primary supply/Operating mine pool - countries.xlsx', sheet_name='Sheet1', index_col=0)
open_parameter=pd.read_excel('Data/primary supply/Opening subsample parameter.xlsx', sheet_name='max5', index_col=0)
incentive_pool=pd.read_excel('Data/primary supply/Incentive mine pool - countries.xlsx', sheet_name='Sheet1', index_col=0)
pri_hyper_param=pd.read_excel('Data/primary supply/Hyperparameters.xlsx', sheet_name='Sheet1', index_col=0)
operating_mine_pool_cn = operating_mine_pool.loc[operating_mine_pool.loc[:, 'Country'] == 'China']
operating_mine_pool_rw = operating_mine_pool.loc[operating_mine_pool.loc[:, 'Country'] != 'China']
incentive_pool_cn = incentive_pool.loc[incentive_pool.loc[:, 'Country'] == 'China']
incentive_pool_rw = incentive_pool.loc[incentive_pool.loc[:, 'Country'] != 'China']

## Refinery parameters
ref_hyper_param=pd.read_excel('Data/refined supply/Refinery hyperparameter.xlsx', sheet_name='Parameters', index_col=0)
ref_hyper_param_cn = pd.read_excel('Data/refined supply/Refinery hyperparameter.xlsx', sheet_name='CN Parameters', index_col=0)
ref_hyper_param_rw = pd.read_excel('Data/refined supply/Refinery hyperparameter.xlsx', sheet_name='RW Parameters', index_col=0)
conc_to_cathode_eff=ref_hyper_param.loc['conc to cathode eff', 'Value']
scrap_to_cathode_eff=ref_hyper_param.loc['scrap to cathode eff', 'Value']
ref_prod_history = historical_prod.loc[1960:, 'Primary refining production':'Refined usage'].copy()
ref_prod_history_cn = historical_prod_cn.loc[1960:, 'Primary refining production':'Refined production, WoodMac'].copy()
ref_prod_history_cn.columns = ref_prod_history.columns
ref_prod_history_rw = ref_prod_history - ref_prod_history_cn
historical_ref_imports_cn = pd.Series(0,index=np.arange(1960,2041))
historical_ref_imports_cn.loc[:2018] = historical_prod_cn.loc[:, 'Net Refined Imports'].copy()
historical_ref_imports_cn.loc[2019:] = historical_ref_imports_cn.loc[2018]
historical_ref_imports_cn.loc[:1974] += historical_prod_cn.loc[:1974,'Semis Imports (COMTRADE, kt)']

## Semis demand parameters
gdp_growth_prediction_base=pd.read_excel('Data/semis demand/Demand prediction data.xlsx', sheet_name='GDP growth', index_col=0, usecols=np.arange(6))
volume_prediction_base=pd.read_excel('Data/semis demand/Demand prediction data.xlsx', sheet_name='All sectors', index_col=0, header=[0,1])
intensity_prediction=pd.read_excel('Data/semis demand/Intensity initial.xls', sheet_name='Sheet1', index_col=0, header=[0,1])
elas_sec_reg=pd.read_excel('Data/semis demand/Elasticity estimates.xlsx', sheet_name='S+R S intercept only', index_col=0)
sector_shape_matrix=pd.read_excel('Data/semis demand/Sector to shape matrix updated.xlsx', sheet_name='Sheet1', index_col=0)
calibration_1718=pd.read_excel('Data/semis demand/2017 and 2018 calibration.xlsx', sheet_name='Sheet1', index_col=0)

# Adjust demand in 2018 to scale it back to ICSG
intensity_prediction.loc[2017, :] = intensity_prediction.loc[2017, :]\
.mul(calibration_1718.loc[2017, 'ICSG refined usage']).div(calibration_1718.loc[2017, 'simulated refined usage'])
intensity_prediction.loc[2018, :] = intensity_prediction.loc[2018, :]\
.mul(calibration_1718.loc[2018, 'ICSG refined usage']).div(calibration_1718.loc[2018, 'simulated refined usage'])
demand_prediction=volume_prediction_base.loc[2015:, :].mul(intensity_prediction.fillna(0))

## Scrap supply parameters
use_sector_combined=pd.read_excel('Data/scrap supply/End use combined data.xlsx', sheet_name='Combined', index_col=0)
sector_to_product=pd.read_excel('Data/scrap supply/All accounting matrix.xlsx', sheet_name='sector to product', index_col=0)
product_to_waste=pd.read_excel('Data/scrap supply/All accounting matrix.xlsx', sheet_name='product to waste', index_col=0)
product_life_and_eff=pd.read_excel('Data/scrap supply/All accounting matrix.xlsx', sheet_name='product lifetime and efficiency', index_col=0)
product_to_cathode_alloy=pd.read_excel('Data/scrap supply/All accounting matrix.xlsx', sheet_name='product to copper or alloy', index_col=0)
recycle_efficiency=pd.read_excel('Data/scrap supply/All accounting matrix.xlsx', sheet_name='recycling efficiency', index_col=0)

# Availability-specific parameters
s2s = pd.read_excel('Data/Shape-Sector Distributions.xlsx', index_col=0)
prod_spec = pd.read_excel('Data/Prod_spec_20200311_no_low.xlsx')
raw_spec = pd.read_excel('Data/Raw_spec_201901.xlsx',index_col=0)
raw_spec.drop(['Grape','Low_brass','Pb_Red_Brass'],inplace=True)
for i in prod_spec.index:
    prod_spec.loc[i,'UNS'] = prod_spec.loc[i,'UNS']+' '+prod_spec.loc[i,'Category']
    
# Home scrap ratio
home_scrap_ratio_file=pd.read_excel('Data/scrap supply/Home scrap ratio.xls', sheet_name='Sheet1', index_col=0)
home_scrap_ratio_series=home_scrap_ratio_file.loc[:, 'Calibrated ratio']
exchange_scrap_ratio_series=0.9-home_scrap_ratio_series

# Sector end use to product matrix 
use_product_history = use_sector_combined.apply(lambda x: (x*sector_to_product).sum(axis=1),axis=1)
demand_fractions = pd.read_excel('Data/semis demand/demand_analysis_copper_lto_q2_2016.xls', sheet_name='Analysis', index_col = 0)
cn_demand_fraction = demand_fractions.loc[:,'China Fraction']
rw_demand_fraction = 1 - cn_demand_fraction
use_product_history_cn = use_product_history.apply(lambda x: x*cn_demand_fraction.loc[:2018])
use_product_history_rw = use_product_history.apply(lambda x: x*rw_demand_fraction.loc[:2018])

# Product to waste matrices
product_to_waste_collectable=product_to_waste.iloc[:, :-2]
product_to_waste_no_loss=product_to_waste_collectable.mul(1/product_to_waste_collectable.sum(axis=1), axis=0)

# Product lifetime parameters and frequencies
product_lifetime=product_life_and_eff.loc[:, 'Lifetime']
product_lifetime_cn = product_life_and_eff.loc[:, 'CN Lifetime']
product_lifetime_df=lifetime_df(product_lifetime)
product_lifetime_df_cn = lifetime_df(product_lifetime_cn)
product_lifetime_freq_df=lifetime_freq_df(product_lifetime_df)
product_lifetime_freq_df_cn = lifetime_freq_df(product_lifetime_df_cn)

# Recycling and fabrication efficiencies
sort_eff=recycle_efficiency.iloc[:, 0]
sort_eff_cn = recycle_efficiency.iloc[:, 2]
collect_rate=recycle_efficiency.iloc[:, 1]
collect_rate_cn = recycle_efficiency.iloc[:, 3]
fab_eff=product_life_and_eff.loc[:, 'Fabrication efficiency']
fab_eff_cn = product_life_and_eff.loc[:, 'CN Fabrication efficiency']
new_scrap_gen=1/fab_eff-1
new_scrap_gen_cn = 1/fab_eff_cn-1
sort_eff_series=pd.DataFrame(np.array((list(sort_eff)*23)).reshape(23, 6), index=np.arange(2018, 2041), columns=sort_eff.index)
sort_eff_series_cn = pd.DataFrame(np.array((list(sort_eff_cn)*23)).reshape(23, 6), index = np.arange(2018,2041), columns = sort_eff_cn.index)
collect_rate_series=pd.DataFrame(np.array((list(collect_rate)*23)).reshape(23, 6), index=np.arange(2018, 2041), columns=collect_rate.index)
collect_rate_series_cn=pd.DataFrame(np.array((list(collect_rate_cn)*23)).reshape(23, 6), index=np.arange(2018, 2041), columns=collect_rate_cn.index)

## Price formation parameters
price_formation_param=pd.read_excel('Data/price formation/Price formation.xlsx', sheet_name='Sheet1', index_col=0)
cathode_sd_elas=price_formation_param.loc['Cathode SD elasticity', 'Value']
conc_sd_elas=price_formation_param.loc['Concentrate SD elasticity', 'Value']
cathode_sp2_elas=price_formation_param.loc['SP2 cathode elasticity', 'Value']
sp2_sd_elas=price_formation_param.loc['SP2 SD elasticity', 'Value']
cathode_sp1_elas=price_formation_param.loc['SP1 cathode elasticity', 'Value']
sp1_sd_elas=price_formation_param.loc['SP1 SD elasticity', 'Value']
cathode_alloyed_elas=price_formation_param.loc['SP2 SD elasticity', 'Value']
alloyed_sd_elas=price_formation_param.loc['SP Alloy SD elasticity', 'Value']

print(str(datetime.now()))

AttributeError: module 'numpy' has no attribute 'asscalar'

# Historical initialization

In [ ]:
system_initialization()
scrap_subset = list(['Yellow_Brass','Cartridge', 'No.1', 'No.2', 'Pb_Yellow_Brass', 'Ocean', 
                    'Red_Brass', 'Al_Bronze', 'Ni_Ag'])
# fraction_yellows is fraction of Yellow_Brass, Pb_Yellow_Brass, and Cartridge allowed in secondary refineries
fraction_yellows = 0.05 
# Scrap correction factor due to accumulation from prior year's unused scrap being available; maintains scrap spread following original simulation pathway
scrap_bal_correction = 0.94
# Post-industrial recycled material price is defined as 0.65 multiplied by No.1 scrap price, as PIR prices most closely correlate with No.1 price and 0.65 allows all PIR to be consumed, aligning with industry expectations
pir_price_set = 0.65

# Initialize simulation time
history_start_time='19600101'
simulation_start_time='20180101'
simulation_end_time='20400101'
simulation_time=pd.date_range(simulation_start_time, simulation_end_time, freq='AS')
history_time=pd.date_range(history_start_time, simulation_start_time, freq='AS')

# Cathode price
cathode_price_series=pd.Series(0, index=history_time)
cathode_price_series.loc[:'20180101']=historical_lme_price.values
cathode_bal_l1 = pd.Series(0, index = np.arange(2018,2041))

# TCRC
tcrc_series=pd.Series(0, index=history_time)
tcrc_series.loc[:'20180101']=historical_tcrc.values

# Raw prices initialization
raw_price_cn = raw_price.copy()
raw_price_rw = raw_price.copy()
scraps = [i for i in raw_price.columns if 'Ref' not in i and 'No.' not in i]

# Scrap spreads (No.2, No.1, alloyed)
sp2_series=pd.Series(0, index=history_time)
sp2_series.loc[:'20180101']=historical_sp2.values
sp2_series_cn=pd.Series(0, index=history_time)
sp2_series_cn.loc[:'20180101']=historical_sp2.values
sp2_series_rw=pd.Series(0, index=history_time)
sp2_series_rw.loc[:'20180101']=historical_sp2.values
sp1_series = pd.Series(0, index=history_time)
sp1_series.loc[:'20180101'] = historical_sp1.values
sp1_series_cn = pd.Series(0, index=history_time)
sp1_series_cn.loc[:'20180101'] = historical_sp1.values
sp1_series_rw = pd.Series(0, index=history_time)
sp1_series_rw.loc[:'20180101'] = historical_sp1.values
spa_series = pd.DataFrame(0, index=history_time, columns=scraps)
spa_series.loc[:'20180101']= historical_spa.values
spa_series_cn = pd.DataFrame(0, index=history_time, columns=scraps)
spa_series_cn.loc[:'20180101']= historical_spa.values
spa_series_rw = pd.DataFrame(0, index=history_time, columns=scraps)
spa_series_rw.loc[:'20180101']= historical_spa.values

# Initialize mining stats
mine_life_stats_panel_operating=mine_life_stats_panel_init(simulation_time, operating_mine_pool)
mine_pool_new_last=pd.DataFrame()
mine_life_stats_panel_new_last=pd.DataFrame()
total_mining_prod=pd.Series(0, index=simulation_time)

# Initilize sxew ids
sxew_id_operating_bool=operating_mine_pool.loc[:, 'Payable percent (%)']==100
sxew_id_operating=[i for i in sxew_id_operating_bool.index if sxew_id_operating_bool.loc[i]]
conc_id_operating_bool=operating_mine_pool.loc[:, 'Payable percent (%)']!=100
conc_id_operating=[i for i in conc_id_operating_bool.index if conc_id_operating_bool.loc[i]]
sxew_id_new_bool=incentive_pool.loc[:, 'Payable percent (%)']==100
sxew_id_new=[i for i in sxew_id_new_bool.index if sxew_id_new_bool.loc[i]]
conc_id_new_bool=incentive_pool.loc[:, 'Payable percent (%)']!=100
conc_id_new=[i for i in conc_id_new_bool.index if conc_id_new_bool.loc[i]]

sxew_new=pd.Series(0, index=simulation_time)
sxew_all=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
sxew_all_cn=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
sxew_all_rw=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
sxew_all.loc['20170101']=historical_prod.loc[2017, 'SX-EW production']
sxew_all_cn.loc['20170101']=historical_prod_cn.loc[2017, 'SX-EW production']
sxew_all_rw.loc['20170101']=historical_prod_rw.loc[2017, 'SX-EW production']


# Initialize refinery stats
ref_stats=ref_stats_init(simulation_time, ref_hyper_param)
ref_stats_cn = ref_stats_init(simulation_time, ref_hyper_param_cn)
ref_stats_rw = ref_stats_init(simulation_time, ref_hyper_param_rw)
ref_bal_l1 = pd.Series(0, index = np.arange(2018,2041))
ref_bal_l1_cn = pd.Series(0, index = np.arange(2018,2041))
ref_bal_l1_rw = pd.Series(0, index = np.arange(2018,2041))

# Initialize concentrate prod, add 2017
conc_prod_series=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
conc_prod_series_cn=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
conc_prod_series_rw=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
conc_prod_series_cn.loc['20170101']=historical_prod_cn.loc[2017, 'Concentrate production']
conc_prod_series_rw.loc['20170101']=historical_prod_rw.loc[2017, 'Concentrate production']
conc_prod_series.loc['20170101'] = conc_prod_series_cn.loc['20170101'] + conc_prod_series_rw.loc['20170101']

# Initialize refined supply and demand
ref_prod_series=pd.Series(0, index=simulation_time)
ref_prod_series_cn=pd.Series(0, index=simulation_time)
ref_prod_series_rw=pd.Series(0, index=simulation_time)
ref_demand_series=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
ref_demand_series_cn=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
ref_demand_series_rw=pd.Series(0, index=pd.date_range('20170101', simulation_end_time, freq='AS'))
ref_demand_series_cn.loc['20170101']=historical_prod_cn.loc[2017, 'Refined usage']# - historical_ref_imports_cn.loc[2017]
ref_demand_series_rw.loc['20170101']=historical_prod_rw.loc[2017, 'Refined usage']# + historical_ref_imports_cn.loc[2017]
ref_demand_series.loc['20170101'] = ref_demand_series_cn.loc['20170101'] + ref_demand_series_rw.loc['20170101']

# Initialize end use by product stats
use_product_future=pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns=use_product_history.columns)
use_product_all_life=pd.concat([use_product_history, use_product_future])
use_product_future_cn=pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns=use_product_history_cn.columns)
use_product_all_life_cn=pd.concat([use_product_history_cn, use_product_future_cn])
use_product_future_rw=pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns=use_product_history_rw.columns)
use_product_all_life_rw=pd.concat([use_product_history_rw, use_product_future_rw])

# Initialize old scrap history
product_eol_history_cn=product_reach_eol(use_product_history_cn, product_lifetime_freq_df_cn)
product_eol_history_rw=product_reach_eol(use_product_history_rw, product_lifetime_freq_df)
product_eol_history=product_eol_history_cn + product_eol_history_rw

# Old scrap available 
old_scrap_available_history_cn = old_scrap_gen_init(product_eol_history_cn, product_to_waste_collectable, product_to_cathode_alloy,
                                    collect_rate_cn, sort_eff_cn, prod_spec.copy(), s2s)
old_scrap_available_history_rw = old_scrap_gen_init(product_eol_history_rw, product_to_waste_collectable, product_to_cathode_alloy,
                                    collect_rate, sort_eff, prod_spec.copy(), s2s)
old_scrap_available_history = old_scrap_available_history_cn + old_scrap_available_history_rw

old_scrap_available_future=pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns=old_scrap_available_history.columns)
old_scrap_available_future_cn=pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns=old_scrap_available_history_cn.columns)
old_scrap_available_future_rw=pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns=old_scrap_available_history_rw.columns)
old_scrap_available_cn = pd.concat([old_scrap_available_history_cn, old_scrap_available_future_cn])
old_scrap_available_rw = pd.concat([old_scrap_available_history_rw, old_scrap_available_future_rw])
old_scrap_available = pd.concat([old_scrap_available_history, old_scrap_available_future])

# Initialize direct melt demand
direct_melt_sectorial_demand_cn=(use_product_all_life_cn*product_to_cathode_alloy.loc[:, 'Alloyed'])
direct_melt_sectorial_demand_rw=(use_product_all_life_rw*product_to_cathode_alloy.loc[:, 'Alloyed'])
direct_melt_sectorial_demand = direct_melt_sectorial_demand_cn + direct_melt_sectorial_demand_rw

direct_melt_sectorial_demand_cn.loc[:,'Unalloyed'] = (use_product_all_life_cn*product_to_cathode_alloy.loc[:, 'Copper']).sum(axis = 1)
direct_melt_sectorial_demand_rw.loc[:,'Unalloyed'] = (use_product_all_life_rw*product_to_cathode_alloy.loc[:, 'Copper']).sum(axis = 1)
direct_melt_sectorial_demand.loc[:, 'Unalloyed'] = direct_melt_sectorial_demand_cn.loc[:, 'Unalloyed'] + direct_melt_sectorial_demand_rw.loc[:, 'Unalloyed']
# Including imports
direct_melt_sectorial_demand_cn.loc[1960:2018, 'Unalloyed'] -= historical_ref_imports_cn.loc[1960:2018]
direct_melt_sectorial_demand_rw.loc[1960:2018, 'Unalloyed'] += historical_ref_imports_cn.loc[1960:2018]

# Initialize new scrap history
new_scrap_available_history_cn = pd.DataFrame(0, index = product_eol_history_cn.index, columns = old_scrap_available_history_cn.columns)
new_scrap_available_history_rw = pd.DataFrame(0, index = product_eol_history_rw.index, columns = old_scrap_available_history_rw.columns)
new_scrap_available_history = pd.DataFrame(0, index = product_eol_history.index, columns = old_scrap_available_history.columns)
new_scrap_alloys_cn = pd.DataFrame(0, index = product_eol_history_cn.index, columns = list(prod_spec.loc[:,'Primary code'].unique())+list(['New No.1']))
new_scrap_alloys_rw = pd.DataFrame(0, index = product_eol_history_rw.index, columns = list(prod_spec.loc[:,'Primary code'].unique())+list(['New No.1']))
prod_spec_cop = prod_spec.copy()
for i in prod_spec_cop.index:
    prod_spec_cop.loc[i+prod_spec.shape[0],:] = prod_spec_cop.loc[i,:]
    prod_spec_cop.loc[i+prod_spec.shape[0],'UNS'] = prod_spec_cop.loc[i,'UNS'] + '_rw'

for year_i in new_scrap_available_history.index:
    home_scrap_ratio=home_scrap_ratio_series.loc[year_i]
    exchange_scrap_ratio=exchange_scrap_ratio_series.loc[year_i]
    
    # Initialize new scrap availability history
    new_scrap_available_year_i_cn, new_scrap_alloys_cn_year_i = \
    new_scrap_gen_oneyear(use_product_history_cn.loc[year_i], product_to_waste_no_loss, product_to_cathode_alloy, 
                        collect_rate_cn, sort_eff_cn, prod_spec.copy(), s2s, new_scrap_gen_cn, exchange_scrap_ratio, 
                        home_scrap_ratio, 1)
    new_scrap_available_year_i_rw, new_scrap_alloys_rw_year_i = \
    new_scrap_gen_oneyear(use_product_history_rw.loc[year_i], product_to_waste_no_loss, product_to_cathode_alloy, 
                        collect_rate, sort_eff, prod_spec.copy(), s2s, new_scrap_gen, exchange_scrap_ratio, 
                        home_scrap_ratio 2)
    new_scrap_alloys_cn_year_i.loc[new_scrap_alloys_cn_year_i.loc[:,'Availability'] < 0,'Availability'] = 0
    new_scrap_alloys_rw_year_i.loc[new_scrap_alloys_rw_year_i.loc[:,'Availability'] < 0,'Availability'] = 0
        
    new_scrap_available_history_cn.loc[year_i] = new_scrap_available_year_i_cn
    new_scrap_available_history_rw.loc[year_i] = new_scrap_available_year_i_rw
    new_scrap_available_history.loc[year_i] = new_scrap_available_year_i_cn + new_scrap_available_year_i_rw
    new_scrap_alloys_cn.loc[year_i] = new_scrap_alloys_cn_year_i.loc[:,'Availability']
    new_scrap_alloys_rw.loc[year_i] = new_scrap_alloys_rw_year_i.loc[:,'Availability']

new_scrap_alloys_compositions = new_scrap_alloys_cn_year_i.loc[:,:'Low_Fe']

new_scrap_available_future_cn = pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns = new_scrap_available_history_cn.columns)
new_scrap_available_future_rw = pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns = new_scrap_available_history_rw.columns)
new_scrap_available_future = pd.DataFrame(0, index=np.arange(2019, 2041, 1), columns = new_scrap_available_history.columns)
new_scrap_available_cn = pd.concat([new_scrap_available_history_cn, new_scrap_available_future_cn])
new_scrap_available_rw = pd.concat([new_scrap_available_history_rw, new_scrap_available_future_rw])
new_scrap_available = pd.concat([new_scrap_available_history, new_scrap_available_future])

# Assumes China scrap distribution is the same as the rest of the world
scrap_imports_cn_all = (old_scrap_available_rw.apply(lambda x: x / old_scrap_available_rw.sum(axis=1) * \
                        historical_prod_cn.loc[:,'Copper Scrap Imports, COMTRADE (kt)'])).loc[1960:2040]

# China's future imports
for i in scrap_imports_cn_all.loc[2019:].index:
    scrap_imports_cn_all.loc[i, :] = scrap_imports_cn_all.loc[2018, :]
original_scrap_imports_cn_all = scrap_imports_cn_all.copy()
    
# Summing to produce all available scraps
all_scrap_available_cn = old_scrap_available_cn + new_scrap_available_cn + scrap_imports_cn_all
all_scrap_available_rw = old_scrap_available_rw + new_scrap_available_rw - scrap_imports_cn_all
all_scrap_available = all_scrap_available_cn + all_scrap_available_rw
total_unalloyed_cn = all_scrap_available_cn.loc[:,'No.1':'No.2'].sum(axis = 1)
total_unalloyed_rw = all_scrap_available_rw.loc[:,'No.1':'No.2'].sum(axis = 1)
total_unalloyed = all_scrap_available.loc[:,'No.1':'No.2'].sum(axis = 1)

# assuming historical No.1/No.2 ratio is at the 2018 value into the past
all_scrap_available_cn.loc[:,'No.1'] = total_unalloyed_cn.loc[:] - historical_prod_cn.loc[:, 'Secondary refining production'] / scrap_to_cathode_eff * 0.85
all_scrap_available_rw.loc[:,'No.1'] = total_unalloyed_rw.loc[:] - historical_prod_rw.loc[:, 'Secondary refining production'] / scrap_to_cathode_eff * 0.85
all_scrap_available_cn.loc[:,'No.2'] = historical_prod_cn.loc[:, 'Secondary refining production'] / scrap_to_cathode_eff * 0.85
all_scrap_available_rw.loc[:,'No.2'] = historical_prod_rw.loc[:, 'Secondary refining production'] / scrap_to_cathode_eff * 0.85
all_scrap_available.loc[:, 'No.1'] = all_scrap_available_cn.loc[:, 'No.1'] + all_scrap_available_rw.loc[:, 'No.1']
all_scrap_available.loc[:, 'No.2'] = all_scrap_available_cn.loc[:, 'No.2'] + all_scrap_available_rw.loc[:, 'No.2']

all_scrap_available_no_accumulation = all_scrap_available.copy() 
all_scrap_available_no_accumulation_cn = all_scrap_available_cn.copy() 
all_scrap_available_no_accumulation_rw = all_scrap_available_rw.copy() 

# Removing new scrap that is now in alloy form
all_scrap_available_cn.loc[1961:,'Al_Bronze':] -= alloy_to_scrap(new_scrap_alloys_cn, new_scrap_alloys_compositions).loc[1961:]
all_scrap_available_rw.loc[1961:,'Al_Bronze':] -= alloy_to_scrap(new_scrap_alloys_rw, new_scrap_alloys_compositions).loc[1961:]
all_scrap_available = all_scrap_available_cn + all_scrap_available_rw
all_scrap_available_no_accumulation_cn.loc[1961:,'Al_Bronze':] -= alloy_to_scrap(new_scrap_alloys_cn, new_scrap_alloys_compositions).loc[1961:]
all_scrap_available_no_accumulation_rw.loc[1961:,'Al_Bronze':] -= alloy_to_scrap(new_scrap_alloys_rw, new_scrap_alloys_compositions).loc[1961:]
all_scrap_available_no_accumulation = all_scrap_available_cn + all_scrap_available_rw


# Initialize scrap demand
direct_melt_demand = pd.DataFrame(0, index = np.arange(1960,2041), columns = list(raw_price.columns) + list(new_scrap_alloys_compositions.index))
direct_melt_demand_cn = pd.DataFrame(0, index = np.arange(1960,2041), columns = list(raw_price.columns) + list(new_scrap_alloys_compositions.index))
direct_melt_demand_rw = pd.DataFrame(0, index = np.arange(1960,2041), columns = list(raw_price.columns) + list(new_scrap_alloys_compositions.index))

refined_scrap_demand_cn=historical_prod_cn.loc[:, 'Secondary refining production'].div(scrap_to_cathode_eff)
refined_scrap_demand_rw=historical_prod_rw.loc[:, 'Secondary refining production'].div(scrap_to_cathode_eff)
refined_scrap_demand = refined_scrap_demand_cn + refined_scrap_demand_rw
sec_ref_scrap_demand = direct_melt_demand.copy() # records scraps used in secondary refining
sec_ref_scrap_demand_cn = direct_melt_demand.copy()
sec_ref_scrap_demand_rw = direct_melt_demand.copy()

# Blending to determine demand in the direct melt sector
for year_i in np.arange(1960,2019):
    if year_i > 1960:
        all_scrap_available_cn.loc[year_i,'Al_Bronze':] += alloy_to_scrap_1yr(new_scrap_alloys_cn.loc[year_i-1] - direct_melt_demand_cn.loc[year_i-1, new_scrap_alloys_cn.columns], new_scrap_alloys_compositions)
        all_scrap_available_rw.loc[year_i,'Al_Bronze':] += alloy_to_scrap_1yr(new_scrap_alloys_rw.loc[year_i-1] - direct_melt_demand_rw.loc[year_i-1, new_scrap_alloys_rw.columns], new_scrap_alloys_compositions)
        
        all_scrap_available_cn.loc[year_i,:] += all_scrap_available_cn.loc[year_i-1,:] - direct_melt_demand_cn.loc[year_i-1,:]
        all_scrap_available_rw.loc[year_i,:] += all_scrap_available_rw.loc[year_i-1,:] - direct_melt_demand_rw.loc[year_i-1,:]
        all_scrap_available.loc[year_i,:] = all_scrap_available_cn.loc[year_i,:] + all_scrap_available_rw.loc[year_i,:] 
    
    new_scrap_alloys_cn_year_i = new_scrap_alloys_compositions.copy().loc[:,'High_Cu':]
    new_scrap_alloys_cn_year_i.loc[:,'Availability'] = new_scrap_alloys_cn.loc[year_i]
    new_scrap_alloys_rw_year_i = new_scrap_alloys_compositions.copy().loc[:,'High_Cu':]
    new_scrap_alloys_rw_year_i.loc[:,'Availability'] = new_scrap_alloys_rw.loc[year_i]
    
    all_scrap_available_year_i = all_scrap_available.loc[year_i]
    all_scrap_available_year_i_cn = all_scrap_available_cn.loc[year_i]
    all_scrap_available_year_i_rw = all_scrap_available_rw.loc[year_i]

    if year_i < 1973: #needed to limit PIR consumption to allow scrap buildup and ensure model solution
        pir_price = 1.09
    else:
        pir_price = pir_price_set
    
    direct_melt_demand_cn.loc[year_i,:], direct_melt_demand_rw.loc[year_i,:], sec_ref_scrap_demand_cn.loc[year_i,:], sec_ref_scrap_demand_rw.loc[year_i,:],  = \
        blend_import_ban(all_scrap_available_year_i_cn, all_scrap_available_year_i_rw,
                        direct_melt_sectorial_demand_cn.loc[year_i], direct_melt_sectorial_demand_rw.loc[year_i],
                        raw_price_cn.loc[year_i], raw_price_rw.loc[year_i], s2s, prod_spec.copy(), raw_spec, 
                        refined_scrap_demand_cn.loc[year_i], refined_scrap_demand_rw.loc[year_i],
                        historical_prod_cn.loc[year_i,'Refined usage'], historical_prod_rw.loc[year_i,'Refined usage'],
                        fraction_yellows,
                        new_scrap_alloys_cn_year_i, new_scrap_alloys_rw_year_i, pir_price = pir_price)
    sec_ref_scrap_demand.loc[year_i,:] = sec_ref_scrap_demand_cn.loc[year_i,:] + sec_ref_scrap_demand_rw.loc[year_i,:]
    direct_melt_demand.loc[year_i, :] = direct_melt_demand_cn.loc[year_i, :] + direct_melt_demand_rw.loc[year_i, :] 
    
    if year_i % 5 == 0:
        print(year_i)

    

total_scrap_demand_all_life=pd.DataFrame({'Direct melt scrap': direct_melt_demand.loc[:, scrap_subset].sum(axis=1) - refined_scrap_demand, \
                                        'Refined scrap': refined_scrap_demand})
total_scrap_demand_all_life_cn=pd.DataFrame({'Direct melt scrap': direct_melt_demand_cn.loc[:, scrap_subset].sum(axis=1) - refined_scrap_demand_cn, \
                                            'Refined scrap': refined_scrap_demand_cn})
total_scrap_demand_all_life_rw=pd.DataFrame({'Direct melt scrap': direct_melt_demand_rw.loc[:, scrap_subset].sum(axis=1) - refined_scrap_demand_rw, \
                                            'Refined scrap': refined_scrap_demand_rw})

print(str(datetime.now()))

In [ ]:
%store new_scrap_available
%store new_scrap_available_cn
%store new_scrap_available_rw

%store new_scrap_alloys_cn
%store new_scrap_alloys_rw

%store new_scrap_alloys_compositions

%store scrap_imports_cn_all
%store original_scrap_imports_cn_all

%store historical_ref_imports_cn
%store original_historical_ref_imports_cn

%store all_scrap_available
%store all_scrap_available_cn
%store all_scrap_available_rw

%store total_unalloyed

%store all_scrap_available_no_accumulation
%store all_scrap_available_no_accumulation_cn
%store all_scrap_available_no_accumulation_rw

%store direct_melt_sectorial_demand
%store direct_melt_sectorial_demand_cn
%store direct_melt_sectorial_demand_rw

%store refined_scrap_demand
%store refined_scrap_demand_cn
%store refined_scrap_demand_rw

%store direct_melt_demand
%store direct_melt_demand_cn
%store direct_melt_demand_rw

%store sec_ref_scrap_demand
%store sec_ref_scrap_demand_cn
%store sec_ref_scrap_demand_rw

%store raw_price
%store raw_price_cn
%store raw_price_rw

%store total_scrap_demand_all_life
%store total_scrap_demand_all_life_cn
%store total_scrap_demand_all_life_rw

%store scrap_use_avail_ratio
%store scrap_use_avail_ratio_cn
%store scrap_use_avail_ratio_rw

%store historical_ref_imports_cn
%store scrap_imports_cn_all

print(str(datetime.now()))

# Multi-Scenario Simulations

In [ ]:
scenarios = ['test']

for scenario in np.arange(0,len(scenarios)):
    print('___________________________________________________________________\nNEW SCENARIO:', scenarios[scenario], '('+str(scenario)+'/'+str(len(scenarios))+')')
    system_initialization()
    %store -r
    
    # Spread difference multiplier
    if 'sdiff' in scenarios[scenario]:
        spread_diff_multiplier = float(scenarios[scenario].split()[1])
    
    
    # Defining scenarios for refined import sensitivities
    for i in np.arange(2019,2041):
        historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018]
        if ' inc' in scenarios[scenario] or '_200' in scenarios[scenario].split()[-1]:
            if  '200' in scenarios[scenario]:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] + (i-2018)*200
            else:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] + (i-2018)*100
        elif ' dec' in scenarios[scenario]:
            if '200' in scenarios[scenario]:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] - (i-2018)*200
            else:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] - (i-2018)*100
    historical_ref_imports_cn.loc[historical_ref_imports_cn<0] = 0

    
    print(str(datetime.now()))
    mine_life_stats_panel_operating=mine_life_stats_panel_init(simulation_time, operating_mine_pool)
    mine_pool_new_last=pd.DataFrame()
    mine_life_stats_panel_new_last=pd.DataFrame()
    volume_prediction_base=pd.read_excel('Data/semis demand/Demand prediction data.xlsx', sheet_name='All sectors', index_col=0, header=[0,1])

    for year_i in np.arange(2018, 2041):

        print('  ', year_i, scenarios[scenario])
        t=pd.datetime(year_i, 1, 1)
        t_lag_1=pd.datetime(year_i-1, 1, 1)
        t_lag_2=pd.datetime(year_i-2, 1, 1)

        #### Scenario parameters ####
        sort_eff=sort_eff_series.loc[year_i]
        collect_rate=collect_rate_series.loc[year_i]
        sort_eff_cn=sort_eff_series_cn.loc[year_i]
        collect_rate_cn=collect_rate_series_cn.loc[year_i]

        #### Price formation ####
        if year_i > 2018:        
            ### Cathode price ###
            cathode_bal_l1.loc[year_i]=ref_prod_series.loc[t_lag_1]-ref_demand_series.loc[t_lag_1]
            print('\tref prod: ', ref_prod_series.loc[t_lag_1], ' ref demand: ', ref_demand_series.loc[t_lag_1])
            cathode_price_series.loc[t]=cathode_price_predict(cathode_price_series.loc[t_lag_1], 
                                                              cathode_balance=cathode_bal_l1.loc[year_i], cathode_sd_elas=cathode_sd_elas)

            ### TCRC ###
            conc_bal_l1.loc[year_i]=conc_prod_series.loc[t_lag_1]-ref_stats.loc[t_lag_1, 'Primary production']/conc_to_cathode_eff
            tcrc_series.loc[t]=tcrc_predict(tcrc_series.loc[t_lag_1], \
                                            conc_balance=conc_bal_l1.loc[year_i], conc_sd_elas=conc_sd_elas)
            tcrc_series_cn.loc[t] = tcrc_series.copy().loc[t]
            tcrc_series_rw.loc[t] = tcrc_series.copy().loc[t]
            
            # Testing TCRC sensitivities, only need to uncomment for those sensitivity tests
#             if tcrc_multiplier_cn != 0 and year_i <= tcrc_years+2018:
#                 if 'cn' in scenarios[scenario].split()[1]:
#                     tcrc_series_cn.loc[t] *= tcrc_multiplier_cn
#                 if 'rw' in scenarios[scenario].split()[1]:
#                     tcrc_series_rw.loc[t] *= tcrc_multiplier_rw
#                 print('\t\tCN TCRC:', tcrc_series_cn.loc[t], 'Global TCRC:', tcrc_series.loc[t])
            
            ### Scrap balances
            print('Scrap balance correction factor:', scrap_bal_correction)
            scrap_bal_l1.loc[year_i] = all_scrap_available_no_accumulation.loc[year_i-1, scrap_subset].sum() - total_scrap_demand_all_life.loc[year_i-1].sum() / scrap_bal_correction
            scrap_bal_l1_cn.loc[year_i] = all_scrap_available_no_accumulation_cn.loc[year_i-1, scrap_subset].sum() - total_scrap_demand_all_life_cn.loc[year_i-1].sum() / scrap_bal_correction
            scrap_bal_l1_rw.loc[year_i] = all_scrap_available_no_accumulation_rw.loc[year_i-1, scrap_subset].sum() - total_scrap_demand_all_life_rw.loc[year_i-1].sum() / scrap_bal_correction
            
            cathode_price_diff=cathode_price_series.loc[t]-cathode_price_series.loc[t_lag_1]
            # sp1 is No.1 scrap spread, sp2 is No.2 scrap spread, spa is alloyed scrap spread
            sp2_series.loc[t]=sp2_predict(sp2_series.loc[t_lag_1], \
                                          scrap_balance=scrap_bal_l1.loc[year_i], scrap_sd_elas=sp2_sd_elas, \
                                          cathode_diff=cathode_price_diff, cathode_sp2_elas=cathode_sp2_elas)
            sp2_series_cn.loc[t]=sp2_predict(sp2_series_cn.loc[t_lag_1], \
                                          scrap_balance=scrap_bal_l1_cn.loc[year_i], scrap_sd_elas=sp2_sd_elas_cn, \
                                          cathode_diff=cathode_price_diff, cathode_sp2_elas=cathode_sp2_elas)
            sp2_series_rw.loc[t]=sp2_predict(sp2_series_rw.loc[t_lag_1], \
                                          scrap_balance=scrap_bal_l1_rw.loc[year_i], scrap_sd_elas=sp2_sd_elas_rw,\
                                          cathode_diff=cathode_price_diff, cathode_sp2_elas=cathode_sp2_elas)
            sp1_series.loc[t]=sp1_predict(sp1_series.loc[t_lag_1],\
                                         scrap_balance=scrap_bal_l1.loc[year_i], scrap_sd_elas=sp1_sd_elas,\
                                         cathode_diff=cathode_price_diff, cathode_sp1_elas=cathode_sp1_elas)
            sp1_series_cn.loc[t]=sp1_predict(sp1_series_cn.loc[t_lag_1],\
                                         scrap_balance=scrap_bal_l1_cn.loc[year_i], scrap_sd_elas=sp1_sd_elas_cn,\
                                         cathode_diff=cathode_price_diff, cathode_sp1_elas=cathode_sp1_elas)
            sp1_series_rw.loc[t]=sp1_predict(sp1_series_rw.loc[t_lag_1],\
                                         scrap_balance=scrap_bal_l1_rw.loc[year_i], scrap_sd_elas=sp1_sd_elas_rw,\
                                         cathode_diff=cathode_price_diff, cathode_sp1_elas=cathode_sp1_elas)
            spa_series.loc[t]=alloy_predict(spa_series.loc[t_lag_1],\
                                         scrap_balance=scrap_bal_l1.loc[year_i], scrap_sd_elas=alloyed_sd_elas,\
                                         cathode_diff=cathode_price_diff, cathode_alloy_elas=cathode_alloyed_elas)
            spa_series_cn.loc[t]=alloy_predict(spa_series_cn.loc[t_lag_1],\
                                         scrap_balance=scrap_bal_l1_cn.loc[year_i], scrap_sd_elas=alloyed_sd_elas_cn,\
                                         cathode_diff=cathode_price_diff, cathode_alloy_elas=cathode_alloyed_elas)
            spa_series_rw.loc[t]=alloy_predict(spa_series_rw.loc[t_lag_1],\
                                         scrap_balance=scrap_bal_l1_rw.loc[year_i], scrap_sd_elas=alloyed_sd_elas_cn,\
                                         cathode_diff=cathode_price_diff, cathode_alloy_elas=cathode_alloyed_elas)
            if 'sdiff' in scenarios[scenario]:
                print('\t\tBefore spread difference multiplier:', spread_diff_multiplier, 'CN:', round(sp2_series_cn.loc[t],2), 'RoW:', round(sp2_series_rw.loc[t],2), 'Global:', round(sp2_series.loc[t],2))
                sp2_series_cn.loc[t] = sp2_series.loc[t] - (sp2_series.loc[t] - sp2_series_cn.loc[t])*spread_diff_multiplier
                sp1_series_cn.loc[t] = sp1_series.loc[t] - (sp1_series.loc[t] - sp1_series_cn.loc[t])*spread_diff_multiplier
                spa_series_cn.loc[t] = spa_series.loc[t] - (spa_series.loc[t] - spa_series_cn.loc[t])*spread_diff_multiplier
                sp2_series_rw.loc[t] = sp2_series.loc[t] - (sp2_series.loc[t] - sp2_series_rw.loc[t])*spread_diff_multiplier
                sp1_series_rw.loc[t] = sp1_series.loc[t] - (sp1_series.loc[t] - sp1_series_rw.loc[t])*spread_diff_multiplier
                spa_series_rw.loc[t] = spa_series.loc[t] - (spa_series.loc[t] - spa_series_rw.loc[t])*spread_diff_multiplier
                print('\t\tAfter spread difference multiplier:', spread_diff_multiplier, 'CN:', round(sp2_series_cn.loc[t],2), 'RoW:', round(sp2_series_rw.loc[t],2), 'Global:', round(sp2_series.loc[t],2))
                
            raw_price.loc[year_i, 'Ref_Cu'] = cathode_price_series.loc[t]
            raw_price_cn.loc[year_i, 'Ref_Cu'] = cathode_price_series.loc[t]
            raw_price_rw.loc[year_i, 'Ref_Cu'] = cathode_price_series.loc[t]
            raw_price.loc[year_i, 'No.1'] = cathode_price_series.loc[t] - sp1_series.loc[t]
            raw_price_cn.loc[year_i, 'No.1'] = cathode_price_series.loc[t] - sp1_series_cn.loc[t]
            raw_price_rw.loc[year_i, 'No.1'] = cathode_price_series.loc[t] - sp1_series_rw.loc[t]
            raw_price.loc[year_i, 'No.2'] = cathode_price_series.loc[t] - sp2_series.loc[t]
            raw_price_cn.loc[year_i, 'No.2'] = cathode_price_series.loc[t] - sp2_series_cn.loc[t]
            raw_price_rw.loc[year_i, 'No.2'] = cathode_price_series.loc[t] - sp2_series_rw.loc[t]
            raw_price.loc[year_i, list(spa_series.columns)] = spa_series.loc[t].apply(lambda x: cathode_price_series.loc[t] - x)
            raw_price_cn.loc[year_i, list(spa_series.columns)] = spa_series_cn.loc[t].apply(lambda x: cathode_price_series.loc[t] - x)
            raw_price_rw.loc[year_i, list(spa_series.columns)] = spa_series_rw.loc[t].apply(lambda x: cathode_price_series.loc[t] - x)

            ### Refined (non-Cu) Metal Prices ### assumes they change proportionally to copper price
            raw_price.loc[year_i,ref_metals] = raw_price.loc[year_i-1,ref_metals].apply(lambda x: x * cathode_price_series[t] / cathode_price_series[t_lag_1])
            raw_price_cn.loc[year_i,ref_metals] = raw_price_cn.loc[year_i-1,ref_metals].apply(lambda x: x * cathode_price_series[t] / cathode_price_series[t_lag_1])
            raw_price_rw.loc[year_i,ref_metals] = raw_price_rw.loc[year_i-1,ref_metals].apply(lambda x: x * cathode_price_series[t] / cathode_price_series[t_lag_1])
            

        # TCRC to cents per pound
        tcrc_cpp_series = tcrc_series.copy().div(22.0462)


        #### Production of operating mine ####
        for mine_id in operating_mine_pool.index:
            mine_data = operating_mine_pool.loc[mine_id]
            mine_life_stats = simulate_mine_life_stats_panel(simulation_time, year_i, 2018, \
                                                             mine_life_stats_panel_operating, mine_data, \
                                                             cathode_price_series, tcrc_cpp_series, \
                                                             pri_hyper_param, \
                                                             mine_cu_pct_change=mine_cu_pct_change.loc[year_i])
            mine_life_stats_panel_operating.loc[:, idx[mine_id, :]]=mine_life_stats.loc[simulation_time, :].values

        # Total production statistics
        prod_operating=mine_life_stats_panel_operating.loc[:, idx[:, 'Recovered metal production (kt)']].sum(axis=1)
        sxew_operating=mine_life_stats_panel_operating.loc[:, idx[sxew_id_operating, 'Recovered metal production (kt)']].sum(axis=1)
        conc_operating=prod_operating-sxew_operating
        print('\tMines closing this year:', (mine_life_stats_panel_operating.loc[t,idx[:,'Ramp flag']] == 'Reclaim').sum())


        #### Production of new mines ####
        if year_i > 2018:
            # Read subsample size parameter and generate new incentive pool for year_i
            subsample_size=open_parameter.loc[year_i, 'Subsample size']
            mine_pool_new_year_i=new_mine_data(year_i, simulation_end_time, incentive_pool, cathode_price_series, tcrc_cpp_series,\
                                               pri_hyper_param, subsample_size, irr_cutoff=0.15)
            print('\tNew mines opening this year: ', mine_pool_new_year_i.shape[0])

            # Initialize mine life panel data for incentive pool year_i
            mine_life_stats_panel_new_year_i=mine_life_stats_panel_init(simulation_time, mine_pool_new_year_i)

            # Append new incentive pool at year_i
            mine_pool_new=pd.concat([mine_pool_new_last, mine_pool_new_year_i])
            mine_life_stats_panel_new=pd.concat([mine_life_stats_panel_new_last, mine_life_stats_panel_new_year_i], axis=1)

            for mine_id in mine_pool_new.index:
                mine_data = mine_pool_new.loc[mine_id]
                mine_life_stats = simulate_mine_life_stats_panel(simulation_time, year_i, mine_data.loc['Initial year'], 
                                                                 mine_life_stats_panel_new, mine_data, 
                                                                 cathode_price_series, tcrc_cpp_series, pri_hyper_param,
                                                                 mine_cu_pct_change=mine_cu_pct_change.loc[year_i])
                mine_life_stats_panel_new.loc[:, idx[mine_id, :]]=mine_life_stats.loc[simulation_time, :].values


            # Total production statistics
            prod_new=mine_life_stats_panel_new.loc[:, idx[:, 'Recovered metal production (kt)']].sum(axis=1)
            sxew_new=mine_life_stats_panel_new.loc[:, idx[sxew_id_new, 'Recovered metal production (kt)']].sum(axis=1)
            conc_new=prod_new-sxew_new

            # Update incentive pool info
            mine_pool_new_last=mine_pool_new.copy()
            mine_life_stats_panel_new_last=mine_life_stats_panel_new.copy()


        #### Update total mining production ####
        if year_i == 2018:
            conc_new=pd.Series(0, index=simulation_time)
            prod_new=pd.Series(0, index=simulation_time)

        conc_prod_series.loc[t]=conc_operating.loc[t]+conc_new.loc[t]
        sxew_all.loc[t]=sxew_operating.loc[t]+sxew_new.loc[t]
        print('\tTotal mining production: ', prod_operating.loc[t]+prod_new.loc[t])


        #### Demand for refined copper ####
        if year_i == 2018:
            pass
        else:    
            intensity_next=intensity_prediction_one_year(year_i, cathode_price_series, gdp_growth_prediction_base, \
                                                         intensity_prediction.loc[year_i-1, :], volume_prediction_base, \
                                                         elas_sec_reg, method='sec and reg')
            intensity_prediction.loc[year_i, :]=intensity_next.values
            demand_prediction.loc[year_i, :]=intensity_prediction.loc[year_i, :].mul(volume_prediction_base.loc[year_i, :]).values

        # Demand by shape and refined copper demand - beginning of China separation
        demand_by_sector_cn = demand_prediction.loc[:,idx[:,'China']].groupby(level=0,axis=1).sum()
        demand_by_sector_rw = demand_prediction.groupby(level=0,axis=1).sum() - demand_prediction.loc[:,idx[:,'China']].groupby(level=0,axis=1).sum()
        demand_by_sector = demand_prediction.groupby(level=0,axis=1).sum()
        
        if year_i > 2018:
            ref_demand_series.loc[t_lag_1]=direct_melt_demand.loc[year_i-1, 'Ref_Cu']
            ref_demand_series_cn.loc[t_lag_1]=direct_melt_demand_cn.loc[year_i-1, 'Ref_Cu'] 
            ref_demand_series_rw.loc[t_lag_1]=direct_melt_demand_rw.loc[year_i-1, 'Ref_Cu']
            ref_demand_series.loc[t_lag_1] = ref_demand_series_cn.loc[t_lag_1] + ref_demand_series_rw.loc[t_lag_1]
        
        #### Production of refineries ####
        if year_i == 2018:
            pass
        else:
            ref_bal_l1.loc[year_i] = ref_prod_series.loc[t_lag_1]/ref_demand_series.loc[t_lag_1]
            ref_bal_l1_cn.loc[year_i] = ref_prod_series_cn.loc[t_lag_1]/ref_demand_series_cn.loc[t_lag_1]
            ref_bal_l1_rw.loc[year_i] = ref_prod_series_rw.loc[t_lag_1]/ref_demand_series_rw.loc[t_lag_1]
            ref_stats_next_cn=simulate_refinery_production_oneyear(year_i, tcrc_series_cn, sp2_series_cn, \
                                                                   ref_demand_series_cn, all_scrap_available_no_accumulation_cn.sum(axis=1),
                                                                   ref_stats_cn, ref_hyper_param_cn, growth_lag=1,
                                                                   ref_cu_pct_change = ref_cu_pct_change.loc[year_i],
                                                                   ref_sr_pct_change = ref_sr_pct_change.loc[year_i])        
            ref_stats_next_rw=simulate_refinery_production_oneyear(year_i, tcrc_series_rw, sp2_series_rw, 
                                                                   ref_demand_series_rw, all_scrap_available_no_accumulation_rw.sum(axis=1),
                                                                   ref_stats_rw, ref_hyper_param_rw, growth_lag=1, 
                                                                   ref_cu_pct_change = ref_cu_pct_change.loc[year_i],
                                                                   ref_sr_pct_change = ref_sr_pct_change.loc[year_i])
            ref_stats_cn.loc[t, :]=ref_stats_next_cn
            ref_stats_rw.loc[t, :]=ref_stats_next_rw
            ref_stats.loc[t,['Primary capacity', 'Secondary capacity', 'Primary production', 'Secondary production']] = \
               ref_stats_next_cn.loc[['Primary capacity', 'Secondary capacity', 'Primary production', 'Secondary production']] \
               + ref_stats_next_rw.loc[['Primary capacity', 'Secondary capacity', 'Primary production', 'Secondary production']]

        total_ref_prod_cn=ref_stats_cn.loc[t, 'Primary production']+ref_stats_cn.loc[t, 'Secondary production']
        total_ref_prod_rw=ref_stats_rw.loc[t, 'Primary production']+ref_stats_rw.loc[t, 'Secondary production']+sxew_all.loc[t]
        total_ref_prod = total_ref_prod_cn + total_ref_prod_rw
        ref_prod_series_cn.loc[t]=total_ref_prod_cn
        ref_prod_series_rw.loc[t]=total_ref_prod_rw
        ref_prod_series.loc[t] = total_ref_prod_cn + total_ref_prod_rw


        #### Generation of old scrap ####
        if year_i > 2018:
            # Calculate end use by sector and by product
            use_sector_year_i_cn=demand_by_sector_cn.loc[year_i]
            use_sector_year_i_rw=demand_by_sector_rw.loc[year_i]
            use_product_year_i_cn=pd.Series(np.matmul(use_sector_year_i_cn, sector_to_product.transpose()), \
                                             index=sector_to_product.index)
            use_product_year_i_rw=pd.Series(np.matmul(use_sector_year_i_rw, sector_to_product.transpose()), \
                                            index=sector_to_product.index)
            use_product_all_life_cn.loc[year_i]=use_product_year_i_cn.values
            use_product_all_life_rw.loc[year_i]=use_product_year_i_rw.values
            use_product_all_life.loc[year_i]=use_product_year_i_cn.values + use_product_year_i_rw.values


            # Product reaching end of life and waste collected
            product_eol_year_i_cn = product_reach_eol_oneyear(year_i, use_product_all_life_cn, product_lifetime_freq_df)
            product_eol_year_i_rw = product_reach_eol_oneyear(year_i, use_product_all_life_rw, product_lifetime_freq_df)
            product_eol_year_i = product_eol_year_i_cn + product_eol_year_i_rw
            
            # Old scrap available by scrap type
            old_scrap_available_year_i_cn = old_scrap_gen_oneyear(product_eol_year_i_cn, product_to_waste_collectable, \
                                                                  product_to_cathode_alloy, collect_rate_cn, sort_eff_cn, \
                                                                  prod_spec, s2s)
            old_scrap_available_year_i_rw = old_scrap_gen_oneyear(product_eol_year_i_rw, product_to_waste_collectable, \
                                                                  product_to_cathode_alloy, collect_rate, sort_eff, \
                                                                  prod_spec, s2s)
            old_scrap_available_cn.loc[year_i] = old_scrap_available_year_i_cn
            old_scrap_available_rw.loc[year_i] = old_scrap_available_year_i_rw
            old_scrap_available.loc[year_i] = old_scrap_available_year_i_cn + old_scrap_available_year_i_rw

        # Defining this segment earlier than before
        direct_melt_sectorial_demand_cn.loc[year_i, :'Diverse']=(use_product_all_life_cn*product_to_cathode_alloy.loc[:, 'Alloyed']).loc[year_i]
        direct_melt_sectorial_demand_rw.loc[year_i, :'Diverse']=(use_product_all_life_rw*product_to_cathode_alloy.loc[:, 'Alloyed']).loc[year_i]
        direct_melt_sectorial_demand.loc[year_i, :'Diverse']=direct_melt_sectorial_demand_cn.loc[year_i,:'Diverse'] + direct_melt_sectorial_demand_rw.loc[year_i,:'Diverse']

        direct_melt_sectorial_demand_cn.loc[year_i,'Unalloyed'] = (use_product_all_life_cn.loc[year_i]*\
                                                                   product_to_cathode_alloy.loc[:, 'Copper']).sum()\
                                                                   - historical_ref_imports_cn.loc[year_i]
        direct_melt_sectorial_demand_rw.loc[year_i,'Unalloyed'] = (use_product_all_life_rw.loc[year_i]*\
                                                                   product_to_cathode_alloy.loc[:, 'Copper']).sum() \
                                                                   + historical_ref_imports_cn.loc[year_i]
        direct_melt_sectorial_demand.loc[year_i, 'Unalloyed'] = direct_melt_sectorial_demand_cn.loc[year_i,'Unalloyed'] + direct_melt_sectorial_demand_rw.loc[year_i,'Unalloyed']

        #### Generation of new scrap ####
        if year_i > 2018:
            home_scrap_ratio=home_scrap_ratio_series.loc[year_i]
            exchange_scrap_ratio=exchange_scrap_ratio_series.loc[year_i]
            
            # New scrap available by scrap type
            new_scrap_available_year_i_cn, new_scrap_alloys_cn_year_i = \
                new_scrap_gen_oneyear(use_product_all_life_cn.loc[year_i], product_to_waste_no_loss, product_to_cathode_alloy, \
                                  collect_rate_cn, sort_eff_cn, prod_spec.copy(), s2s, new_scrap_gen_cn, exchange_scrap_ratio, \
                                  home_scrap_ratio)
            new_scrap_available_year_i_rw, new_scrap_alloys_rw_year_i = \
                new_scrap_gen_oneyear(use_product_all_life_rw.loc[year_i], product_to_waste_no_loss, product_to_cathode_alloy, \
                                  collect_rate, sort_eff, prod_spec.copy(), s2s, new_scrap_gen, exchange_scrap_ratio, \
                                  home_scrap_ratio)
            new_scrap_available_cn.loc[year_i] = new_scrap_available_year_i_cn
            new_scrap_available_rw.loc[year_i] = new_scrap_available_year_i_rw
            new_scrap_available.loc[year_i] = new_scrap_available_year_i_cn + new_scrap_available_year_i_rw

            new_scrap_alloys_cn.loc[year_i] = new_scrap_alloys_cn_year_i.loc[:,'Availability']
            new_scrap_alloys_rw.loc[year_i] = new_scrap_alloys_rw_year_i.loc[:,'Availability']
            new_scrap_alloys_compositions = new_scrap_alloys_cn_year_i.loc[:,:'Low_Fe']
                
            # All scrap
            all_scrap_available_cn.loc[year_i] = old_scrap_available_year_i_cn + new_scrap_available_year_i_cn
            all_scrap_available_rw.loc[year_i] = old_scrap_available_year_i_rw + new_scrap_available_year_i_rw
            all_scrap_available.loc[year_i] = all_scrap_available_cn.loc[year_i] + all_scrap_available_rw.loc[year_i]
            all_scrap_available_cn.loc[year_i] += scrap_imports_cn_all.loc[year_i]
            all_scrap_available_rw.loc[year_i] -= scrap_imports_cn_all.loc[year_i]

        total_unalloyed_year_i_cn = all_scrap_available_cn.loc[year_i,'No.1':'No.2'].sum()
        total_unalloyed_year_i_rw = all_scrap_available_rw.loc[year_i,'No.1':'No.2'].sum()
        total_unalloyed_year_i = all_scrap_available.loc[year_i,'No.1':'No.2'].sum()

        print('\tTotal scrap supply, China: ', all_scrap_available_cn.sum(axis=1).loc[year_i])
        print('\tTotal scrap supply, RoW: ', all_scrap_available_rw.sum(axis=1).loc[year_i])

        # Updating scrap availability
        if year_i > 2018:
            refined_scrap_demand_year_i_cn=ref_stats_cn.loc[t, 'Secondary production']/scrap_to_cathode_eff
            all_scrap_available_cn.loc[year_i, 'No.1'] = total_unalloyed_year_i_cn - refined_scrap_demand_year_i_cn * 0.85
            all_scrap_available_cn.loc[year_i, 'No.2'] = refined_scrap_demand_year_i_cn * 0.85

            refined_scrap_demand_year_i_rw=ref_stats_rw.loc[t, 'Secondary production']/scrap_to_cathode_eff
            all_scrap_available_rw.loc[year_i, 'No.1'] = total_unalloyed_year_i_rw - refined_scrap_demand_year_i_rw * 0.85
            all_scrap_available_rw.loc[year_i, 'No.2'] = refined_scrap_demand_year_i_rw * 0.85

            refined_scrap_demand_year_i = refined_scrap_demand_year_i_cn + refined_scrap_demand_year_i_rw
            all_scrap_available.loc[year_i, 'No.1'] = total_unalloyed_year_i - refined_scrap_demand_year_i * 0.85
            all_scrap_available.loc[year_i, 'No.2'] = refined_scrap_demand_year_i * 0.85
                

            all_scrap_available_no_accumulation.loc[year_i] = all_scrap_available.loc[year_i].copy()
            all_scrap_available.loc[year_i,:] += all_scrap_available.loc[year_i-1,:] - direct_melt_demand.loc[year_i-1,:]
            all_scrap_available_no_accumulation_cn.loc[year_i] = all_scrap_available_cn.loc[year_i].copy()
            all_scrap_available_cn.loc[year_i,:] += all_scrap_available_cn.loc[year_i-1,:] - direct_melt_demand_cn.loc[year_i-1,:]
            all_scrap_available_no_accumulation_rw.loc[year_i] = all_scrap_available_rw.loc[year_i].copy()
            all_scrap_available_rw.loc[year_i,:] += all_scrap_available_rw.loc[year_i-1,:] - direct_melt_demand_rw.loc[year_i-1,:]

            
            all_scrap_available_cn.loc[year_i,'Al_Bronze':] -= alloy_to_scrap_1yr(new_scrap_alloys_cn.loc[year_i], new_scrap_alloys_compositions)
            all_scrap_available_rw.loc[year_i,'Al_Bronze':] -= alloy_to_scrap_1yr(new_scrap_alloys_rw.loc[year_i], new_scrap_alloys_compositions)
            all_scrap_available_cn.loc[year_i,'Al_Bronze':] += alloy_to_scrap_1yr(new_scrap_alloys_cn.loc[year_i-1] - direct_melt_demand_cn.loc[year_i-1, new_scrap_alloys_cn.columns], new_scrap_alloys_compositions)
            all_scrap_available_rw.loc[year_i,'Al_Bronze':] += alloy_to_scrap_1yr(new_scrap_alloys_rw.loc[year_i-1] - direct_melt_demand_rw.loc[year_i-1, new_scrap_alloys_rw.columns], new_scrap_alloys_compositions)
            all_scrap_available.loc[year_i] = all_scrap_available_cn.loc[year_i] + all_scrap_available_rw.loc[year_i]
            all_scrap_available_no_accumulation_cn.loc[year_i,'Al_Bronze':] -= alloy_to_scrap_1yr(new_scrap_alloys_cn.loc[year_i], new_scrap_alloys_compositions)
            all_scrap_available_no_accumulation_rw.loc[year_i,'Al_Bronze':] -= alloy_to_scrap_1yr(new_scrap_alloys_rw.loc[year_i], new_scrap_alloys_compositions)
            all_scrap_available_no_accumulation.loc[year_i] = all_scrap_available_no_accumulation_cn.loc[year_i] + all_scrap_available_no_accumulation_rw.loc[year_i]


            new_scrap_alloys_cn_year_i = new_scrap_alloys_compositions.copy().loc[:,'High_Cu':]
            new_scrap_alloys_cn_year_i.loc[:,'Availability'] = new_scrap_alloys_cn.loc[year_i]
            new_scrap_alloys_rw_year_i = new_scrap_alloys_compositions.copy().loc[:,'High_Cu':]
            new_scrap_alloys_rw_year_i.loc[:,'Availability'] = new_scrap_alloys_rw.loc[year_i]

            
            all_scrap_available_year_i_cn = all_scrap_available_cn.loc[year_i]
            all_scrap_available_year_i_rw = all_scrap_available_rw.loc[year_i]
        
        # Scrap demand
        if year_i > 2018:
            refined_scrap_demand_year_i_cn=ref_stats_cn.loc[t, 'Secondary production']/scrap_to_cathode_eff
            refined_scrap_demand_year_i_rw=ref_stats_rw.loc[t, 'Secondary production']/scrap_to_cathode_eff
            refined_scrap_demand_year_i = refined_scrap_demand_year_i_cn + refined_scrap_demand_year_i_rw
            print('\t\tPIR price:', pir_price, 'times refined copper price')
            
            direct_melt_demand_cn.loc[year_i,:], direct_melt_demand_rw.loc[year_i,:], \
            sec_ref_scrap_demand_cn.loc[year_i,:], sec_ref_scrap_demand_rw.loc[year_i,:] = \
                blend_import_ban(all_scrap_available_year_i_cn, all_scrap_available_year_i_rw,\
                        direct_melt_sectorial_demand_cn.loc[year_i], direct_melt_sectorial_demand_rw.loc[year_i],\
                        raw_price_cn.loc[year_i], raw_price_rw.loc[year_i], s2s, prod_spec.copy(), raw_spec.copy(), \
                        refined_scrap_demand_year_i_cn, refined_scrap_demand_year_i_rw,\
                        direct_melt_sectorial_demand_cn.loc[year_i,'Unalloyed'], direct_melt_sectorial_demand_rw.loc[year_i,'Unalloyed'],\
                        fraction_yellows,\
                        new_scrap_alloys_cn_year_i, new_scrap_alloys_rw_year_i, 
                        pir_price = pir_price)
            sec_ref_scrap_demand.loc[year_i,:] = sec_ref_scrap_demand_cn.loc[year_i,:] + sec_ref_scrap_demand_rw.loc[year_i,:]
        
            direct_melt_demand.loc[year_i,:] = direct_melt_demand_cn.loc[year_i,:] + direct_melt_demand_rw.loc[year_i,:]
            print('\tTotal direct melt scrap demand, China: ', direct_melt_demand_cn.sum(axis=1).loc[year_i])
            print('\tTotal direct melt scrap demand, RoW: ', direct_melt_demand_rw.sum(axis=1).loc[year_i])
            
            total_scrap_demand_all_life_cn.loc[year_i,'Direct melt scrap'] = (direct_melt_demand_cn.loc[year_i,:] - sec_ref_scrap_demand_cn.loc[year_i, :]).loc[scrap_subset].sum() + direct_melt_demand_cn.loc[year_i,'New No.1']
            total_scrap_demand_all_life_cn.loc[year_i,'Refined scrap'] = sec_ref_scrap_demand_cn.loc[year_i,scrap_subset].sum()
            total_scrap_demand_all_life_rw.loc[year_i,'Direct melt scrap'] = (direct_melt_demand_rw.loc[year_i,:] - sec_ref_scrap_demand_rw.loc[year_i, :]).loc[scrap_subset].sum() + direct_melt_demand_rw.loc[year_i,'New No.1']
            total_scrap_demand_all_life_rw.loc[year_i,'Refined scrap'] = sec_ref_scrap_demand_rw.loc[year_i,scrap_subset].sum()
            total_scrap_demand_all_life.loc[year_i,'Direct melt scrap'] = total_scrap_demand_all_life_cn.loc[year_i,'Direct melt scrap'] + total_scrap_demand_all_life_rw.loc[year_i,'Direct melt scrap']
            total_scrap_demand_all_life.loc[year_i,'Refined scrap'] = total_scrap_demand_all_life_cn.loc[year_i,'Refined scrap'] + total_scrap_demand_all_life_rw.loc[year_i,'Refined scrap']
       
        ref_demand_series_cn.loc[t]=direct_melt_demand_cn.loc[year_i, 'Ref_Cu']
        ref_demand_series_rw.loc[t]=direct_melt_demand_rw.loc[year_i, 'Ref_Cu']
        ref_demand_series.loc[t] = ref_demand_series_cn.loc[t] + ref_demand_series_rw.loc[t]
        
    print(str(datetime.now()))

    

# Previously-run scenarios

In [ ]:
for i in ['Easiest scrap import control']:
    # Easiest scrap import control
    import_multiplier = 1
    if ' 50yoy' in scenarios[scenario] or '50_' in scenarios[scenario].split()[-1]:
        import_multiplier = 0.5
    elif ' 75yoy' in scenarios[scenario]:
        import_multiplier = 0.75
    elif ' 25yoy' in scenarios[scenario]:
        import_multiplier = 0.25
    print('Import multiplier:',import_multiplier)
    
    if ' baseline' in scenarios[scenario]:
        scrap_imports_cn_all = original_scrap_imports_cn_all.copy()
           
    elif ' No.2 ' in scenarios[scenario] or 'n_' in scenarios[scenario].split()[-1] or 'no2' in scenarios[scenario]:
        scrap_imports_cn_all = original_scrap_imports_cn_all.copy()
        for i in np.arange(2019,2041):
            scrap_imports_cn_all.loc[i,'No.2'] = scrap_imports_cn_all.loc[i-1,'No.2'] * import_multiplier
                
    elif ' alloyed ' in scenarios[scenario]:
        scrap_imports_cn_all = original_scrap_imports_cn_all.copy()
        for i in np.arange(2019,2041):
            scrap_imports_cn_all.loc[i,'Al_Bronze':] = scrap_imports_cn_all.loc[i-1,'Al_Bronze':] * import_multiplier
                
    elif ' both ' in scenarios[scenario] or 'b_' in scenarios[scenario].split()[-1]:
        scrap_imports_cn_all = original_scrap_imports_cn_all.copy()
        for i in np.arange(2019,2041):
            scrap_imports_cn_all.loc[i,'No.2':] = scrap_imports_cn_all.loc[i-1,'No.2':] * import_multiplier
    else:
        scrap_imports_cn_all = original_scrap_imports_cn_all.copy()
    print('Scrap imports:',scrap_imports_cn_all.loc[2018:])

for i in ['Defining scenarios for refined import sensitivities']:
    for i in np.arange(2019,2041):
        historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018]
        if ' inc ' in scenarios[scenario]:
            if  ' 200 ' in scenarios[scenario]:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] + (i-2018)*200
            else:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] + (i-2018)*100
        elif ' dec ' in scenarios[scenario]:
            if ' 200 ' in scenarios[scenario]:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] - (i-2018)*200
            else:
                historical_ref_imports_cn.loc[i] = historical_ref_imports_cn.loc[2018] - (i-2018)*100
    historical_ref_imports_cn.loc[historical_ref_imports_cn<0] = 0
    print(historical_ref_imports_cn)
    
    

# scrap sd elasticity sensitivities
    scenarios = ['scrap sd elas baseline 20200527',
                 'scrap 1.0 both 50yoy',
                 'scrap 1.0 both 50yoy 100 inc',
                 'scrap 1.0 both 50yoy 200 inc',
                 'scrap 1.0 both 50yoy 100 dec',
                 'scrap 1.0 both 50yoy 200 dec',
                 'scrap 1.1',
                 'scrap 0.9',
                 'scrap 0.8',
                 'scrap 0.7']
    if scenario > 0:
            sp2_sd_elas_cn *= float(scenarios[scenario].split()[1])
            sp2_sd_elas_rw *= float(scenarios[scenario].split()[1])
            sp1_sd_elas_cn *= float(scenarios[scenario].split()[1])
            sp1_sd_elas_rw *= float(scenarios[scenario].split()[1])
            alloyed_sd_elas_cn *= float(scenarios[scenario].split()[1])
            alloyed_sd_elas_rw *= float(scenarios[scenario].split()[1])
            
            

for i in ['TCRC Changes']:
    scenarios = ['regional tcrc baseline 20200616',
             'tcrc cn 1.0 both 50yoy 20200616',
             'tcrc cn 1.0 both 50yoy 100 inc 20200616',
             'tcrc cn 1.0 both 50yoy 200 inc 20200616',
             'tcrc cn 1.0 both 50yoy 100 dec 20200616',
             'tcrc cn 1.0 both 50yoy 200 dec 20200616',
             'tcrc cn 0.95 3 yr',
             'tcrc cn 0.9 3 yr',
             'tcrc cn 0.8 3 yr',
             'tcrc cn 0.7 3 yr']
    tcrc_years = 23
        if scenario > 5:
            tcrc_multiplier_cn = float(scenarios[scenario].split()[2])
            tcrc_multiplier_rw = 0
            if scenarios[scenario].split()[-1] == 'yr':
                tcrc_years = float(scenarios[scenario].split()[3])

                        
for i in ['Scrap SD elasticity']:
    scenarios = ['scrap sd elas baseline',
             'scrap 0.9',
             'scrap 0.8',
             'scrap 0.7',
             'scrap 1.1',
             'scrap 0.9 no2 50yoy',
             'scrap 0.9 no2 50yoy 100 inc']
    
    sp2_sd_elas=price_formation_param.copy().loc['SP2 SD elasticity', 'Value']
    sp1_sd_elas=price_formation_param.copy().loc['SP1 SD elasticity', 'Value']
    alloyed_sd_elas=price_formation_param.copy().loc['SP Alloy SD elasticity', 'Value']
    print('Initial sp2_sd_elas:',sp2_sd_elas)
    if scenario > 0:
        sp2_sd_elas *= float(scenarios[scenario].split()[1])
        sp1_sd_elas *= float(scenarios[scenario].split()[1])
        alloyed_sd_elas *= float(scenarios[scenario].split()[1])

        sp2_sd_elas_cn = sp2_sd_elas
        sp2_sd_elas_rw = sp2_sd_elas
        sp1_sd_elas_cn = sp1_sd_elas
        sp1_sd_elas_rw = sp1_sd_elas
        alloyed_sd_elas_cn = alloyed_sd_elas
        alloyed_sd_elas_rw = alloyed_sd_elas
        print('New sp2_sd_elas:',sp2_sd_elas)
    
for i in ['COVID scenarios']:
        # COVID Changes: Baseline shocks: minecu2.62, refcu:2.29, gdp2.45
    mine_cu_pct_change = pd.Series(0, index=np.arange(2018,2041))
    ref_cu_pct_change = pd.Series(0, index=np.arange(2018,2041))
    ref_sr_pct_change = pd.Series(0, index=np.arange(2018,2041))
    if 'refcu' in scenarios[scenario]:
        ref_cu_pct_change.loc[2020] = float(scenarios[scenario].split('refcu')[-1].split()[0])
        if 'nsr' not in scenarios[scenario]:
            ref_sr_pct_change.loc[2020] = ref_cu_pct_change.loc[2020] * 6.93/2.29 # ratio of 2019 SR and CU changes from ICSG (see 9/4/2020 notes)
        print('Ref CU changed:',ref_cu_pct_change.loc[2020])
        print('Ref SR changed:',ref_sr_pct_change.loc[2020])
    if 'minecu' in scenarios[scenario]:
        mine_cu_ratio_2019_2020 = 6.51/2.62 # since normally would give relative to baseline 2020 production instead of 2019 production
        mine_cu_pct_change.loc[2020] = float(scenarios[scenario].split('minecu')[-1].split()[0]) * mine_cu_ratio_2019_2020 # positive gives increase
        print('Mine CU changed:',mine_cu_pct_change.loc[2020])
        
    if 'gdpc' in scenarios[scenario]:
        gdp_pct_change = float(scenarios[scenario].split('gdpc')[-1].split()[0])
        gdp_growth_prediction_base = pd.read_excel('Data/semis demand/Demand prediction data.xlsx', sheet_name='GDP growth', index_col=0, usecols='BM:BR')
        gdp_growth_prediction_base.columns = ['China', 'EU', 'Japan', 'NAM', 'ROW']
        gdp_growth_prediction_base.loc[2020] = gdp_growth_prediction_base.loc[2019]+(gdp_growth_prediction_base.loc[2020]-gdp_growth_prediction_base.loc[2019])*gdp_pct_change/2.45
        gdp_growth_prediction_base.loc[2021] = gdp_growth_prediction_base.loc[2019]+(gdp_growth_prediction_base.loc[2021]-gdp_growth_prediction_base.loc[2019])*gdp_pct_change/2.45
        print('COVID scenario,',gdp_growth_prediction_base)
    elif 'allshock' in scenarios[scenario]:
        gdp_pct_change = 2.45
        gdp_growth_prediction_base = pd.read_excel('Data/semis demand/Demand prediction data.xlsx', sheet_name='GDP growth', index_col=0, usecols='BM:BR')
        gdp_growth_prediction_base.columns = ['China', 'EU', 'Japan', 'NAM', 'ROW']
        gdp_growth_prediction_base.loc[2020] = gdp_growth_prediction_base.loc[2019]+(gdp_growth_prediction_base.loc[2020]-gdp_growth_prediction_base.loc[2019])*gdp_pct_change/2.45
        gdp_growth_prediction_base.loc[2021] = gdp_growth_prediction_base.loc[2019]+(gdp_growth_prediction_base.loc[2021]-gdp_growth_prediction_base.loc[2019])*gdp_pct_change/2.45
        mine_cu_ratio_2019_2020 = 6.51/2.62 # since normally would give relative to baseline 2020 production instead of 2019 production
        mine_cu_pct_change.loc[2020] = 2.62 * mine_cu_ratio_2019_2020 # positive gives increase
        ref_cu_pct_change.loc[2020] = 2.29
        if 'nsr' not in scenarios[scenario]:
            ref_sr_pct_change.loc[2020] = ref_cu_pct_change.loc[2020] * 6.93/2.29 # ratio of 2019 SR and CU changes from ICSG (see 9/4/2020 notes)
    else:
        gdp_growth_prediction_base = pd.read_excel('Data/semis demand/Demand prediction data.xlsx', sheet_name='GDP growth', index_col=0, usecols=np.arange(6))
